In [ ]:
import process_copus
import numpy as np
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import scipy.io
%matplotlib inline

In [ ]:
nb_id = 1736319231
import time
try:
    print(nb_id)
except NameError:
    print(round(time.time()))

In [ ]:
def closest_index(arr, val):
    if val > arr.max():
        raise ValueError("not in range: {} > {}".format(val, arr.max()))
    elif val < arr.min():
        raise ValueError("not in range: {} < {}".format(val, arr.min()))
    index = np.argmin(abs(arr - val))  # TODO: use binary search or something else?
    return index

In [ ]:
def get_txt_filename(path_raw):
    path = os.path.abspath(path_raw)
    root, ext = os.path.splitext(path)
    txt_filename = root + '.txt'
    return txt_filename

In [ ]:
def out_path(filename):
    global outdir
    path = os.path.join(outdir, filename)
    return path

In [ ]:
def get_filename_prefix(filepath):
    filename = os.path.basename(filepath)
    root, ext = os.path.splitext(filename)
    return root

In [ ]:
def get_outdir(filepath_raw):
    filepath = os.path.abspath(filepath_raw)
    head1, tail1 = os.path.split(filepath)
    return tail1

In [ ]:
def get_topdir_base(filepath_raw):
    filepath = os.path.abspath(filepath_raw)
    head1, tail1 = os.path.split(filepath)
    root, ext = os.path.splitext(tail1)
    return root

In [ ]:
def get_i_from_path(filepath):
    filename = os.path.basename(filepath)
    root, ext = os.path.splitext(filename)
    last_part = root.split('_')[-1]
    index = int(last_part)
    return index

In [ ]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    else:
        raise ValueError

In [ ]:
def get_i2_from_path(filepath):
    filename = os.path.basename(filepath)
    root, ext = os.path.splitext(filename)
    index_str = remove_prefix(root, 'm_full')
    index = int(index_str)
    return index

In [ ]:
def unwrap_1x1(arr2d):
    d1, d2 = arr2d.shape
    if d1 == 1 and d2 == 1:
        arr1d, = arr2d
        unwrapped, = arr1d
    else:
        raise ValueError("cannot cast 2D array with shape '{}'".format(arr2d.shape))
    return unwrapped

In [ ]:
def reduce_identical_arr(l):
    """
    Return the first value of a list
    provided all the values are equal
    and are numpy arrays.
    """
    val1 =  l[0]
    all_same = all([np.array_equal(x, val1) for x in l])
    if all_same == True:
        return val1
    else:
        raise ValueError("list has disparate values")

In [ ]:
def reduce_identical_array_vals(d):
    """
    Return the first value of a dict
    provided all the values are equal
    and are numpy arrays.
    """
    val1 = next(iter(d.values()))
    all_same = all([np.array_equal(d[key], val1) for key in d.keys()])
    if all_same == True:
        return val1
    else:
        raise ValueError("dict has disparate values")


In [ ]:
class MyInfo:
    # Give names of class members.
    def __repr__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'
    def __str__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'

In [ ]:
dirpath_raw = '../fegab_isofreq_out/06_fegab/'
dirpath = os.path.abspath(dirpath_raw)
i_list = {}
for filepath in glob.glob(os.path.join(dirpath, '*.out')):
    print(filepath)
    i = get_i_from_path(filepath)
    i_list[i] = []
    for filepath2 in glob.glob(os.path.join(filepath, 'm_full*.npy')):
#         print(filepath2)
        i2 = get_i2_from_path(filepath2)
        i_list[i].append(i2)
    print(i_list[i])

In [ ]:
n_files = len(os.listdir(dirpath))
chosen_step = [8 for _ in range(n_files)]
# skip_list = [4,5,6,7,8,9]
skip_list = []
outdir = get_outdir(dirpath)
filename_prefix = os.path.basename(dirpath)
subtitle_prefix = os.path.basename(dirpath)
info = {}
for filepath in glob.glob(os.path.join(dirpath, '*.out')):
    i = get_i_from_path(filepath)
    if i in skip_list:
        continue
    info[i] = MyInfo()
    info[i].chosen_step = chosen_step[i]
    info[i].npy_path = os.path.join(filepath, 'm_full{:06d}.npy'.format(chosen_step[i]))
    print(info[i].npy_path)
    M1_raw = np.load(info[i].npy_path)
    info[i].M1 = M1_raw.squeeze()
    info[i].txt_path = os.path.join(dirpath, get_txt_filename(filepath))
    with open(info[i].txt_path) as fp:
        params = process_copus.parse_logfile(fp.readlines())
    Nx = int(params['Nx'])
    Ny = int(params['Ny'])
    dx = params['c']
    dy = params['c']
    xpos = np.linspace(0, Nx, Nx)*dx
    ypos = np.linspace(0, Ny, Ny)*dy
    info[i].params = params
    info[i].xpos = xpos
    info[i].ypos = ypos
    info[i].Nx = Nx
    info[i].Ny = Ny
    info[i].dx = dx
    info[i].dy = dy
    del params, Nx, Ny, dx, dy, xpos, ypos

## Do FFT

In [ ]:
for i, d in info.items():
    M1z = d.M1[2]
    M1z_fft_complex = np.fft.fftshift(np.fft.fft2(M1z))
    M1z_fft = np.abs(M1z_fft_complex)
    kx = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(d.Nx, d=d.dx))
    ky = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(d.Ny, d=d.dy))
    d.M1z_fft = M1z_fft
    d.kx = kx
    d.ky = ky
    del M1z, M1z_fft_complex, M1z_fft, kx, ky

In [ ]:
if not os.path.isdir(outdir):
    os.makedirs(outdir, exist_ok=True)

In [ ]:
GHz = 1e9
ps = 1e-12
nm = 1e-9
um = 1e-6

In [ ]:
kx_fftz_list = []
ky_fftz_list = []
kx_list = []
ky_list = []
freq_list = []
dt_list = []
maxdt_list = []
for i, d in info.items():
    ky_0i = closest_index(d.ky, 0.0)
    kx_fftz_list.append(d.M1z_fft[ky_0i])
    del ky_0i
    kx_0i = closest_index(d.kx, 0.0)
    ky_fftz_list.append(d.M1z_fft[:,kx_0i])
    del kx_0i
    kx_list.append(d.kx)
    ky_list.append(d.ky)
    freq_list.append(d.params['f'])
    dt_list.append(d.params['tstep'])
    maxdt_list.append(d.params['maxdt'])
dt_all = reduce_identical_arr(dt_list)
maxdt_all = reduce_identical_arr(maxdt_list)
kx_all = reduce_identical_arr(kx_list)
ky_all = reduce_identical_arr(ky_list)
kx_fftz = np.stack(kx_fftz_list)
ky_fftz = np.stack(ky_fftz_list)
freq = np.array(freq_list)
t_elapsed_list = [dt*i for i, dt in zip(chosen_step, dt_list)]
t_elapsed = np.array(t_elapsed_list)
t_elapsed_all = reduce_identical_arr(t_elapsed_list)

In [ ]:
t_elapsed/ps

In [ ]:
freq/GHz

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(1.5*6.4, 1.5*4.8)
)
plot = ax.pcolormesh(
    kx_all*um,
    freq/GHz,
    kx_fftz,
    cmap='magma',
    shading='nearest',
);
ax.set_xlabel('$k_x$ [1/um]')
ax.set_ylabel('freq [GHz]')
ax.set_xlim(0, 40)
fig.colorbar(mappable=plot, ax=ax, label="FFT magnitude for $M_z$ [A.U.]");
ax.set_title(subtitle_prefix + ", t = {:.1f} ps, maxdt = {} ps, nb_id = {}".format(t_elapsed_all/ps, maxdt_all/ps, nb_id), fontsize=10)
fig.suptitle("dispersion for $M_z$ in $k_x$ direction");

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_dispersion_kx_fftz.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(1.5*6.4, 1.5*4.8)
)
plot = ax.pcolormesh(
    ky_all*um,
    freq/GHz,
    ky_fftz,
    cmap='magma',
    shading='nearest',
);
ax.set_xlabel('$k_y$ [1/um]')
ax.set_ylabel('freq [GHz]')
ax.set_xlim(0, 10)
fig.colorbar(mappable=plot, ax=ax, label="FFT magnitude for $M_z$ [A.U.]");
# ax.legend(loc='lower right');
ax.set_title(subtitle_prefix + ", t = {:.1f} ps, maxdt = {} ps, nb_id = {}".format(t_elapsed_all/ps, maxdt_all/ps, nb_id), fontsize=10)
fig.suptitle("dispersion for $M_z$ in $k_y$ direction");

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_dispersion_ky_fftz.png"), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
# this_t_elapsed = info[INDEX].params['tstep']*INDEX

In [ ]:
INDEX = 20
xmid = (info[INDEX].xpos.min() + info[INDEX].xpos.max())/2
ymid = (info[INDEX].ypos.min() + info[INDEX].ypos.max())/2
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
norm = matplotlib.colors.TwoSlopeNorm(vmin=-2e-1, vcenter=0, vmax=2e-1)
plot = ax.pcolormesh(
    info[INDEX].xpos/um,
    info[INDEX].ypos/um,
    info[INDEX].M1[2],
    cmap='bwr',
#     vmin=-20,
#     vmax=-100,
    norm = norm,
    shading='nearest',
#     norm=matplotlib.colors.LogNorm(),
);
dx = 2
dy = 2
# ax.set_xlim(xmid/um - dx, xmid/um + dx)
# ax.set_ylim(ymid/um - dy, ymid/um + dy)
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
fig.colorbar(mappable=plot, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", t = {:.1f} ps, maxdt = {} ps, nb_id = {}".format(t_elapsed_all/ps, maxdt_all/ps, nb_id), fontsize=10)
fig.suptitle("$M_z$, f = {:.1f} GHz".format(freq[INDEX]/GHz));

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_M1z_{}.png".format(INDEX)), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
INDEX = 20
xmid = (info[INDEX].xpos.min() + info[INDEX].xpos.max())/2
ymid = (info[INDEX].ypos.min() + info[INDEX].ypos.max())/2
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
blackwhite_cm = matplotlib.colors.LinearSegmentedColormap.from_list('black_white', ['black', 'white'], N=2)
plot = ax.pcolormesh(
    info[INDEX].xpos/um,
    info[INDEX].ypos/um,
    np.sign(info[INDEX].M1[2]),
    cmap=blackwhite_cm,
    shading='nearest',
);
dx = 0.5
dy = 0.5
# ax.set_xlim(xmid/um - dx, xmid/um + dx)
# ax.set_ylim(ymid/um - dy, ymid/um + dy)
ax.set_xlabel('x position [um]')
ax.set_ylabel('y position [um]')
fig.colorbar(mappable=plot, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", t = {:.1f} ps, maxdt = {} ps, nb_id = {}".format(t_elapsed_all/ps, maxdt_all/ps, nb_id), fontsize=10)
fig.suptitle("$M_z$, f = {:.1f} GHz".format(freq[INDEX]/GHz));

In [ ]:
fig.savefig(out_path(filename_prefix + "_heatmap_bw_M1z_{}.png".format(INDEX)), bbox_inches='tight', facecolor="w", dpi=300);

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
# import scipy
# to_save = {}
# to_save.update(params)
# to_save.update({
#     'dirpath': dirpath,
#     'npy_filename': npy_filename,
#     'xpos': xpos,
#     'ypos': ypos,
#     'Mx': M1[0],
#     'My': M1[1],
#     'Mz': M1[2],
#     'kx': kx,
#     'ky': ky,
#     'Mz_fft': M1_fftz,
#     'outdir': outdir,
#     'filename_prefix': filename_prefix,
#     'title_prefix': title_prefix,
#     'subtitle_prefix': subtitle_prefix,
#     'nb_id': nb_id,
# })

# scipy.io.savemat(
#     os.path.join(get_parent_dir(dirpath), get_topdir_base(dirpath) + '_with_fft.mat'),
#     to_save,
#     long_field_names=True,
#     do_compression=True,
# )